In [ ]:
from pathlib import WindowsPath, Path
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
from collections import Counter
import re
from nltk.stem import PorterStemmer
import os
from collections import Counter, OrderedDict

In [ ]:
def frequency_word_per_url(content):
    #given a url returns a dictionary: key-> word, value-> frequency of word
    #words in dictionary are stemmed

    #https://docs.python.org/3/library/re.html used to with re.findall to find all valid words in url
    #https://www.geeksforgeeks.org/python-stemming-words-with-nltk/ used for learining PorterStemmer
    # resp = urlopen(url)
    # html = resp.read()

    soup = BeautifulSoup(content, 'html.parser')
    for cur in soup(['script', 'style']):
        cur.extract()

    text = soup.get_text(separator=" ")
    words = re.findall(r"\b\w+\b", text.lower())

    stemmer = PorterStemmer()
    stemmed_word_lis = [stemmer.stem(word) for word in words]

    frequency = Counter(stemmed_word_lis)

    return dict(frequency)

In [ ]:
def write_to_file(myDict: dict, filename: str):
    with open(filename, 'a') as file:
        for key in myDict:
            data = {key: myDict[key]}
            json.dump(data, file)
            file.write("\n")
    
    print("Finished writing to file", filename)

In [ ]:
def rewrite_to_file(myDict: dict, filename: str):

    with open(filename, 'w') as file:
        for key in myDict:
            pos = file.tell()
            data = {key: myDict[key]}
            json.dump(data, file)
            file.write("\n")
    
    print("Finished writing to file", filename)

In [ ]:
def create_inverted_index(file_name, index_dir):

    key = 0 # for assigning document id
    url_id_map = dict({})   # map each url to an id
    inverted_index  = dict({})
    max_length = 100000
    n_index = 0
    n_id = 0
    
    # read each JSON file, and add url to the map
    with open(file_name, "r") as file:
        for file_path in file:
            file_path = WindowsPath(file_path.strip("\n"))
            print("Reading content from", file_path)
            
            with open(file_path, "r") as file:
                
                # get a json object
                jsonObj = json.load(file)
        
                # get the url and tokens (including its frequency). 
                url = jsonObj["url"]
                content = jsonObj["content"]
                tokens = frequency_word_per_url(content)    # key: token, val: frequency
                
                # map each url to a document id
                url_id_map[key] = url
        
                # update the posting list for each token
                for token in tokens:
                    if token not in inverted_index:
                        inverted_index[token] = {key: tokens[token]}
                    else:
                        inverted_index[token][key] = tokens[token]
                
                if len(inverted_index) >= max_length:
                    inverted_index = dict(sorted(inverted_index.items()))
                    filename = index_dir / f"inverted_index_{str(n_index)}.jsonl" 
                    write_to_file(inverted_index, filename)
                    n_index += 1
                    inverted_index.clear()

                if len(url_id_map) >= max_length:
                    filename = index_dir / f"url_id_{str(n_id)}.jsonl"
                    write_to_file(url_id_map, filename)
                    n_id += 1
                    url_id_map.clear()

        
                # update doc id
                key += 1

        if len(inverted_index) > 0:
            inverted_index = dict(sorted(inverted_index.items()))
            filename = index_dir / f"inverted_index_{str(n_index)}.jsonl"
            write_to_file(inverted_index, filename)
            n_index += 1

        if len(url_id_map) > 0:
            filename = index_dir / f"url_id_{str(n_id)}.jsonl"
            write_to_file(url_id_map, filename)
            n_id += 1
    
    return {"key": key, "n_id": n_id, "n_index": n_index}

In [ ]:
def read_inverted_index(filename):
    inverted_index = dict({})
    
    with open(filename, "r") as file:
        for jsonObj in file:
            jsonObj = json.loads(jsonObj)
            token = list(jsonObj.keys())[0]
            inverted_index[token] = jsonObj[token]
    
    return inverted_index

In [ ]:
def update_url_map(url_map: OrderedDict, filename):
    with open(filename, "r") as file:
        for jsonObj in file:
            jsonObj = json.loads(jsonObj)
            token = list(jsonObj.keys())[0]
            if len(url_map) > 100000:
                url_map.popitem()
            url_map[token] = jsonObj[token]
    return url_map

In [ ]:
def merge_index(filename):
    inverted_index = dict({})
    
    with open(filename, "r") as file:
        for jsonObj in file:
            jsonObj = json.loads(jsonObj)
            token = list(jsonObj.keys())[0]
            if token not in inverted_index:
                inverted_index[token] = jsonObj[token]
            else:
                existed_postings = Counter(inverted_index[token])
                new_postings = Counter(jsonObj[token])
                updated_postings = dict(existed_postings + new_postings)
                inverted_index[token] = updated_postings
    
    return inverted_index

In [ ]:
def group_index(filename):
    grouped_index = []
    alpha_list = [] # contain all the first letters of all current tokens
    temp_inverted_index = dict()
    last_token = ""

    # read inverted_index from file
    print("\n Read index from file", filename)
    inverted_index = read_inverted_index(filename)

    # given a sorted dict, group index by first letter
    for token in inverted_index:
        if last_token == "" or token[0] == last_token[0]:
            temp_inverted_index[token] = inverted_index[token]
        else:
            grouped_index.append(temp_inverted_index.copy())
            alpha_list.append(last_token[0])

            # reset
            temp_inverted_index = dict({})
            temp_inverted_index[token] = inverted_index[token]
        
        last_token = token

    # save the remaining to grouped_index
    if len(temp_inverted_index) > 0:
        grouped_index.append(temp_inverted_index.copy())
        alpha_list.append(last_token[0])
    
    # write index to new files
    index_dir = Path("inverted_index") # folder's name to save index files
    index_dir.mkdir(exist_ok=True)
    for i in range(len(grouped_index)):
        char = alpha_list[i]
        filename = index_dir / f"{char}.jsonl"
        write_to_file(grouped_index[i], filename)

    return alpha_list

In [ ]:
def update_cache_index(cache_inverted_index: OrderedDict, token):
    filename = "offset/" + token[0] + ".jsonl"
    with open(filename, "r") as file:
        for jsonObj in file:
            jsonObj = json.loads(jsonObj)
            term = list(jsonObj.keys())[0]

            if len(cache_inverted_index) > 100000:
                cache_inverted_index.popitem()
            cache_inverted_index[term] = jsonObj[term]
    return cache_inverted_index

In [ ]:
def get_posting(token, pos):
    filename = "final_inverted_index/" + token[0] + ".jsonl"
    posting = {}
    with open(filename, 'r') as file:
        file.seek(pos)
        jsonObj = file.readline()
        jsonObj = json.loads(jsonObj)
        posting = jsonObj[token]
    return posting


In [ ]:
# build a search engine
from nltk.stem import PorterStemmer
import math

def search_engine(queries, total_urls):

    stemmer = PorterStemmer()
    max_length = 100000
    cache_offset_index = OrderedDict()    # recently searched items are moved to the back

    # perform the search
    # cristina lopes, machine learning, ACM, master of software engineering

    tokens = queries.split() # ex: "machine learning" => ['machine', 'learning']
    stemmed_tokens = [stemmer.stem(t) for t in tokens]
    posting_lists = []      # for saving all the url id 
    inverted_index = dict() # for calculating tf-idf score purpose

    # find the posting lists of each token
    for token in stemmed_tokens:
        if token not in cache_offset_index:
            cache_offset_index = update_cache_index(cache_offset_index, token)

        if token in cache_offset_index:
            posting = get_posting(token, int(cache_offset_index[token]))
            posting_lists.append(posting)
            cache_offset_index.move_to_end(token)
            inverted_index[token] = posting

        # Boolean AND: intersect all posting lists
        url_id_set = set()
        for posting_list in posting_lists:
            doc_ids = set(posting_list.keys())
            if not url_id_set:
                url_id_set = doc_ids
            else:
                url_id_set &= doc_ids

        # compute tf-idf scores
        doc_scores = {}
        N = int(total_urls)

        for doc_id in url_id_set:
            score = 0.0
            for token in stemmed_tokens:
                posting = inverted_index.get(token, {})
                tf = posting.get(doc_id, 0)
                df = len(posting) if posting else 1
                idf = math.log(N / (1 + df))
                score += tf * idf
            doc_scores[doc_id] = score
            
        ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)

        return url_id_set, ranked_docs


In [ ]:
# run collect_file_path.py to get the txt file (named "JSON_file_path.txt")
file_name = "JSON_file_path.txt"

# create inverted_index, return number of index files and url_id files
index_dir = Path("jsonl_files") # folder's name to save index files
index_dir.mkdir(exist_ok=True) 
return_dict = create_inverted_index(file_name, index_dir)
# return_dict = {"key": 55393, "n_id": 1, "n_index": 28}  # for developing mode

# save return values to a file
write_to_file(return_dict, "return_values.txt")

# read files in json_files, group index by first letters, return a list of letters
# new inverted_index files are under inverted_index folder
num_of_index_files = return_dict["n_index"]
chars = set([])
for i in range(num_of_index_files):
    filename = index_dir / f"inverted_index_{i}.jsonl"
    letters = group_index(filename)
    chars.update(letters)

# for each inverted_index/{char} file
# reorganzie, merge/update the token postings
index_dir = Path("final_inverted_index") # folder's name to save index files
index_dir.mkdir(exist_ok=True) 
for char in chars:
    filename = "inverted_index/" + char + ".jsonl"
    newFilename = "final_" + filename
    inverted_index = merge_index(filename)
    rewrite_to_file(inverted_index, newFilename)

In [ ]:
# improve the indexer by saving the position of each token in final_inverted_index' files
offset_dir = Path("offset") # folder's name to save index files
offset_dir.mkdir(exist_ok=True)

for filename in Path("final_inverted_index").iterdir():
    position_dict = dict()
    with open(filename, 'r') as file:
        while True:
            pos = file.tell()
            line = file.readline()
            if not line: break

            jsonObj = json.loads(line)
            token = list(jsonObj.keys())[0]
            position_dict[token] = pos
    
    offset_filename = offset_dir / str(filename.name)
    with open(offset_filename, 'a') as file:
        for token in position_dict:
            data = {token: position_dict[token]}
            json.dump(data, file)
            file.write("\n")

In [ ]:
filename = f"jsonl_files/url_id_0.jsonl"
url_map = OrderedDict()
update_url_map(url_map, filename)

In [ ]:
import time
num_of_urls = return_dict["key"]
# num_of_urls = 55393 (for dev mode)

# perform the search
url_map = dict()
while True:

    # ask for input/queries
    queries = input("Input queries (type exit to stop searching): ")
    if queries == "exit".lower():
        break
    else:
        startTime = time.process_time_ns()
        
        url_id_set, ranked_docs = search_engine(queries, num_of_urls)

        # get url_id map
        for id in url_id_set:
            if id not in url_map:
                filename = f"jsonl_files/url_id_{int(id) // 100000}.jsonl"
                url_map = update_url_map(url_map, filename)

        endTime = time.process_time_ns()

        # Output up to 5 URLs (no ranking yet)
        print(f"\nURLs that contain: \"{queries}\"")
        for i, (doc_id, score) in enumerate(ranked_docs[:5]):
            url = url_map.get(doc_id, doc_id)
            print(f"{i+1}. {url} (tf-idf: {score:.4f})")
        print("Time Response:", (endTime-startTime) / 10**6, "ms")
        print()